In [1]:
import pandas as pd

# Use Cases

<h3>Aggregation Methods</h3>

In [2]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar'],
                   'B' : [1, 2, 3, 4, 5, 6],
                   'C' : [2.0, 5., 8., 1., 2., 9.]})
df

,A,B,C
0,foo,1,2.0
1,bar,2,5.0
2,foo,3,8.0
3,bar,4,1.0
4,foo,5,2.0
5,bar,6,9.0


In [3]:
df.groupby('A')['C'].sum()

A
bar    15.0
foo    12.0
Name: C, dtype: float64

<h3>Filter methods</h3>

In [4]:
df.groupby('A').filter(lambda x: x['B'].mean() > 3)

,A,B,C
1,bar,2,5.0
3,bar,4,1.0
5,bar,6,9.0


<h3>Transformation Methods</h3>

In [5]:
df.groupby('A').transform(lambda x: (x - x.mean()))

,B,C
0,-2.0,-2.0
1,-2.0,0.0
2,0.0,4.0
3,0.0,-4.0
4,2.0,-2.0
5,2.0,4.0


<h3>Meta Methods</h3>

In [6]:
groups = df.groupby('A')
groups.groups

{'bar': [1, 3, 5], 'foo': [0, 2, 4]}

<h1>U.S Congress Dataset</h1>

In [7]:
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}

In [8]:
# When wrapping a dictionary in a list, it returns the keys
list(dtypes)

['first_name', 'gender', 'type', 'state', 'party']

In [9]:
df = pd.read_csv('legislators-historical.csv',
                dtype=dtypes,
                usecols=list(dtypes) + ['birthday', 'last_name'],
                parse_dates=['birthday'])
df.head()

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN


In [10]:
df.dtypes

last_name             object
first_name          category
birthday      datetime64[ns]
gender              category
type                category
state               category
party               category
dtype: object

In [11]:
df.shape

(12055, 7)

In [12]:
name_by_states = df.groupby('state')['last_name'].count()
print(name_by_states.shape)
name_by_states.head()

(58,)


state
AK     17
AL    209
AR    117
AS      2
AZ     49
Name: last_name, dtype: int64

<h3>Grouped on Multiple Columns</h3>

In [13]:
df.groupby(["state", "gender"])["last_name"].count()[:5]

state  gender
AK     F           0
       M          17
AL     F           4
       M         205
AR     F           5
Name: last_name, dtype: int64

In [14]:
df.groupby(["state", "gender"])["last_name"].count().index[:5]

MultiIndex([('AK', 'F'),
            ('AK', 'M'),
            ('AL', 'F'),
            ('AL', 'M'),
            ('AR', 'F')],
           names=['state', 'gender'])

In [15]:
 df.groupby(["state", "gender"], as_index=False)["last_name"].count()[:5]

,state,gender,last_name
0,AK,F,0
1,AK,M,17
2,AL,F,4
3,AL,M,205
4,AR,F,5


In [16]:
 df.groupby(["state", "gender"], as_index=False, sort=False)["last_name"].count()[:5]

,state,gender,last_name
0,DE,M,97
1,DE,F,0
2,VA,M,429
3,VA,F,4
4,SC,M,246


<h3>How Groupby Works?</h3>

In [17]:
# It returns DataFrameGroupBy object
by_state = df.groupby('state')
by_state

In [18]:
# It returns a dictionary with "state" as keys and a sequence of index locations for the rows belong to the group as values.
by_state.groups["PA"]

Int64Index([    4,    19,    21,    27,    38,    57,    69,    76,    84,
               88,
            ...
            11838, 11862, 11871, 11873, 11883, 11887, 11926, 11938, 11952,
            11965],
           dtype='int64', length=1053)

In [19]:
# You can retrieve the subtable from a single group.
by_state.get_group('PA')

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11887,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11926,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11938,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11952,Costello,Ryan,1976-09-07,M,rep,PA,Republican


In [20]:
# It is an iterable.
for state, frame in by_state:
    print(f'{state}: {frame["last_name"].count()}')

AK: 17
AL: 209
AR: 117
AS: 2
AZ: 49
CA: 368
CO: 92
CT: 240
DC: 2
DE: 97
DK: 9
FL: 161
GA: 317
GU: 4
HI: 24
IA: 205
ID: 59
IL: 488
IN: 343
KS: 143
KY: 373
LA: 199
MA: 427
MD: 305
ME: 175
MI: 296
MN: 162
MO: 334
MS: 155
MT: 53
NC: 356
ND: 44
NE: 128
NH: 181
NJ: 359
NM: 57
NV: 56
NY: 1468
OH: 676
OK: 93
OL: 2
OR: 90
PA: 1053
PI: 13
PR: 19
RI: 107
SC: 251
SD: 51
TN: 301
TX: 264
UT: 55
VA: 433
VI: 4
VT: 115
WA: 96
WI: 198
WV: 120
WY: 40


<h1>Air Quality Dataset</h1>

In [21]:
df = pd.read_csv('AirQualityUCI.csv', sep=";")
df.shape

(9471, 17)

In [23]:
df = pd.read_csv('AirQualityUCI.csv',
                 sep=";",
                 parse_dates=[['Date', 'Time']],
                 na_values=['-200,0', '-200'],
                 usecols=["Date", "Time", "CO(GT)", "T", "RH", "AH"]
                )

# drop the rows without timestamp
df = df.drop(df[df['Date_Time'] == 'nan nan'].index)

# convert the timestamp string to datetime object
df.Date_Time = pd.to_datetime(df.Date_Time, format='%d/%m/%Y %H.%M.%S', errors='ignore')

# rename the collumns
df.rename(
    columns={
        "CO(GT)": "co",
        "Date_Time": "tstamp",
        "T": "temp_c",
        "RH": "rel_hum",
        "AH": "abs_hum",
        },
    inplace=True
)

# choose one column as an index
df = df.set_index('tstamp')
df.shape

(9357, 4)

In [24]:
df.head(5)

,co,temp_c,rel_hum,abs_hum
tstamp,,,,
2004-03-10 18:00:00,"2,6","13,6","48,9","0,7578"
2004-03-10 19:00:00,2,"13,3","47,7","0,7255"
2004-03-10 20:00:00,"2,2","11,9","54,0","0,7502"
2004-03-10 21:00:00,"2,2","11,0","60,0","0,7867"
2004-03-10 22:00:00,"1,6","11,2","59,6","0,7888"


In [25]:
df.index.max()

Timestamp('2005-04-04 14:00:00')

<h3>Grouping on Derived Arrays</h3>

In [26]:
day_names = df.index.day_name()
day_names[:10]

Index(['Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Thursday', 'Thursday', 'Thursday', 'Thursday'],
      dtype='object', name='tstamp')

In [27]:
# convert all strings in Dataframe to floats
for col in df.columns:
    df[col] = df[col].str.replace(',', '.').astype(float)

In [28]:
df.groupby(day_names)["co"].mean()

tstamp
Friday       2.543041
Monday       2.016741
Saturday     1.861077
Sunday       1.438069
Thursday     2.455505
Tuesday      2.382267
Wednesday    2.400787
Name: co, dtype: float64

In [29]:
hr = df.index.hour
df.groupby([day_names, hr])["co"].mean().rename_axis(["dow", "hr"])

dow        hr
Friday     0     1.936170
           1     1.608511
           2     1.172340
           3     0.887234
           4     0.823333
                   ...   
Wednesday  19    4.146809
           20    3.844681
           21    2.897872
           22    2.102128
           23    1.938298
Name: co, Length: 168, dtype: float64

<h3>Discretizing Continuous Variable into Bins</h3>

In [30]:
bins = pd.cut(df["temp_c"], bins=3, labels=("cool", "warm", "hot"))
bins

tstamp
2004-03-10 18:00:00    cool
2004-03-10 19:00:00    cool
2004-03-10 20:00:00    cool
2004-03-10 21:00:00    cool
2004-03-10 22:00:00    cool
                       ... 
2005-04-04 10:00:00    warm
2005-04-04 11:00:00    warm
2005-04-04 12:00:00    warm
2005-04-04 13:00:00    warm
2005-04-04 14:00:00    warm
Name: temp_c, Length: 9357, dtype: category
Categories (3, object): ['cool' < 'warm' < 'hot']

In [31]:
df[["rel_hum", "abs_hum"]].groupby(bins).agg(["mean", "median"])

rel_hum          abs_hum        
             mean median      mean  median
temp_c                                    
cool    57.651452   59.2  0.665874  0.6581
warm    49.382716   49.3  1.182894  1.1452
hot     24.994334   24.1  1.292958  1.2742

<h3>Resampling: Time-based Groupby</h3>

In [32]:
df.groupby([df.index.year, df.index.quarter])["co"].agg(
    ["max", "min"]
).rename_axis(["year", "quarter"])

max  min
year quarter           
2004 1         8.1  0.3
     2         7.3  0.1
     3         7.5  0.1
     4        11.9  0.1
2005 1         8.7  0.1
     2         5.0  0.3

In [33]:
df.resample("M")["co"].agg(["max", "min"])

,max,min
tstamp,,
2004-03-31,8.1,0.3
2004-04-30,7.3,0.3
2004-05-31,6.5,0.1
2004-06-30,6.4,0.1
2004-07-31,5.3,0.1
2004-08-31,3.5,0.1
2004-09-30,7.5,0.2
2004-10-31,9.5,0.4
2004-11-30,11.9,0.1


<h1>News Aggregator Dataset</h1>

In [34]:
pd.read_csv("newsCorpora.csv", sep="\t", header=None).head(5)

,0,1,2,3,4,5,6,7
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


In [35]:
def parse_millisecond_timestamp(ts):
    """Convert ms since Unix epoch to UTC datetime instance."""
    return pd.to_datetime(ts, unit="ms")

In [36]:
df = pd.read_csv(
    "newsCorpora.csv",
    sep="\t",
    header=None,
    index_col=0,
    names=["title", "url", "outlet", "category", "cluster", "host", "tstamp"],
    parse_dates=["tstamp"],
    date_parser=parse_millisecond_timestamp,
    dtype={
        "outlet": "category",
        "category": "category",
        "cluster": "category",
        "host": "category",
    },
)

df.head(5)

,title,url,outlet,category,cluster,host,tstamp
1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,2014-03-10 16:52:50.698
2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,2014-03-10 16:52:51.207
3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,2014-03-10 16:52:51.550
4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,2014-03-10 16:52:51.793
5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,2014-03-10 16:52:52.027


In [37]:
titles = df.groupby('outlet', sort=False)['title']

In [38]:
titles.head(2)

1         Fed official says weak data caused by weather,...
2         Fed's Charles Plosser sees high bar for change...
3         US open: Stocks fall after Fed official hints ...
4         Fed risks falling 'behind the curve', Charles ...
5         Fed's Plosser: Nasty Weather Has Curbed Job Gr...
                                ...                        
422726    Catholic schools, dioceses determine how to ta...
422769        Ebola cases could eventually reach 20 000: UN
422822                         Ebola death toll tops 1, 550
422850    Ebola crisis controversy clouds AFCON 2015 qua...
422858    Sierra Leone - Into the Hot Zone (HD) - 23' mi...
Name: title, Length: 19598, dtype: object

In [39]:
%%time
df.groupby("outlet", sort=False)["title"].apply(lambda ser: ser.str.contains("Fed").sum()).nlargest(10)

CPU times: user 5.29 s, sys: 0 ns, total: 5.29 s
Wall time: 5.29 s


outlet
Reuters                         161
NASDAQ                          103
Businessweek                     93
Investing.com                    66
Wall Street Journal \(blog\)     61
MarketWatch                      56
Moneynews                        55
Bloomberg                        53
GlobalPost                       51
Economic Times                   44
Name: title, dtype: int64

In [40]:
df["title"].str.contains("Fed")

1          True
2          True
3          True
4          True
5          True
          ...  
422933    False
422934    False
422935    False
422936    False
422937    False
Name: title, Length: 422419, dtype: bool

In [41]:
%%time
df["title"].str.contains("Fed").groupby(df['outlet'], sort=False).sum().nlargest(10)

CPU times: user 251 ms, sys: 0 ns, total: 251 ms
Wall time: 247 ms


outlet
Reuters                         161
NASDAQ                          103
Businessweek                     93
Investing.com                    66
Wall Street Journal \(blog\)     61
MarketWatch                      56
Moneynews                        55
Bloomberg                        53
GlobalPost                       51
Economic Times                   44
Name: title, dtype: int64